In [ ]:
!pip install sacrebleu
!pip install evaluate

import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np
def compute_metrics(eval_preds):
    print(len(eval_preds))
    print(eval_preds)

    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = processor.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
training_args = TrainingArguments(
    output_dir=f"{model_name}-MedEdefics",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=40,
    eval_steps=20,
    logging_steps=20,
    max_steps=40,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["labels"],
    load_best_model_at_end=True,
    report_to=None,
    optim="paged_adamw_8bit",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=train_ds,
    compute_metrics=compute_metrics,
)

trainer.evaluate()
#trainer.train()

In [ ]:
!pip install uptrain sqlite_utils

In [ ]:
import openai
from openai import OpenAI
import pandas as pd

GPT_API_KEY = 

# Function to translate a single sentence
client = OpenAI(
  api_key=GPT_API_KEY,
)


def generate_gpt(prompt):
    completion = client.chat.completions.create(
      model="gpt-4-turbo-preview",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "user", "content": prompt}
      ],
      temperature=0,
      logprobs=True,
    )

    return completion.choices[0].message.content, completion.choices[0].logprobs

In [ ]:
def get_prompt(question, response, ground_truth):
    Prompt = f"""
    ###Task Description:
    You will be provided with a potential answer along with a ground truth answer to a given question. Your task is to evaluate the potential answer if it’s matching the ground truth answer.
    Evaluation Steps:
    1. Write a detailed feedback that assess the matching of the response strictly based on the given score rubric, not evaluating in general.
    2. After writing a feedback, Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Score Rubrics.
    3. The output format should look as follows: [RESULT] (an integer number between 1 and 5)\"
    4. Please do not generate any other opening, closing, and explanations.
    ###The instruction to evaluate:
    {question}
    ###Response to evaluate:
    {response}
    ###Reference Answer:
    {ground_truth}
    ###Score Rubrics:
    Score (from 1 to 5): “Response Match: Evaluate how much the evaluated answer matches the true answer.”
    Score (from 1 to 5): “Completeness: Evaluate if the evaluated answer covers all key aspects mentioned in the true answer. ”
    ###Feedback (scores ONLY): 
    the output should be in the json format like this :
    (Response Match: score from 1 to 5)
    (Completeness: score from 1 to 5)
    """
    return Prompt

In [ ]:
import json
data = {
        "question" : "What is the formula of chlorophyll",
        'response' : 'Chlorophyll is the pigmet used in photosynthesis, it helpes in generating oxygen. C55H72MgN4O5',
        'ground_truth' : 'The formula of chlorophyll is C55H72MgN4O5'
    }

prompt = get_prompt(data['question'], data['response'], data['ground_truth'])
feedback, proba = generate_gpt(prompt)
print(feedback)


In [ ]:
def compute_score(feedback, probas):
    feedback = eval(feedback)
    probs = []
    for token in proba.content:
        try:
            token_value = int(token.token)
            probs.append(np.exp(token.logprob))
        except ValueError:
            pass
        
    score1 = feedback["Response Match"]
    score2 = feedback["Completeness"]
    scores = [score1, score2]
    weighted_scores = [score * prob for score, prob in zip(scores, probs)]
    final_weighted_score = np.mean(weighted_scores)
    return final_weighted_score

In [ ]:
def G_eval(data):
    scores = [compute_score(feedback, probas) for feedback, probas in [generate_gpt(get_prompt(data['question'], data['response'], data['ground_truth'])) for data in data]]
    return scores

In [ ]:
good_data = [
    {
        "question" : "What is the formula of chlorophyll",
        'response' : 'Chlorophyll a pigment usuall found in the leaves has the formula C55H72MgN4O5.',
        'ground_truth' : 'The formula of chlorophyll is C55H72MgN4O5'
    }
]

bad_data = [
    {
        "question" : "What is the formula of chlorophyll",
        'response' : 'Chlorophyll is the pigmet used in photosynthesis, it helpes in generating oxygen',
        'ground_truth' : 'The formula of chlorophyll is C55H72MgN4O5'
    }
]

data = good_data + bad_data
G_eval(data)

In [ ]:
# import torch
# from transformers import AutoTokenizer, LlamaForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# model = LlamaForCausalLM.from_pretrained("kaist-ai/Prometheus-7b-v1.0", device_map="auto")


In [ ]:
from uptrain import EvalLLM, ResponseMatching
import json

OPENAI_API_KEY =


good_data = [
    {
        "question" : "What is the formula of chlorophyll",
        'response' : 'Chlorophyll a pigment usuall found in the leaves has the formula C55H72MgN4O5.',
        'ground_truth' : 'The formula of chlorophyll is C55H72MgN4O5'
    }
]

bad_data = [
    {
        "question" : "What is the formula of chlorophyll",
        'response' : 'Chlorophyll is the pigmet used in photosynthesis, it helpes in generating oxygen',
        'ground_truth' : 'The formula of chlorophyll is C55H72MgN4O5'
    }
]

data = good_data + bad_data

eval_llm = EvalLLM(openai_api_key=OPENAI_API_KEY)

res = eval_llm.evaluate(
    data = data,
    checks = [ResponseMatching(method = 'llm')]    # method: llm/exact/rouge
)

In [ ]:
print(json.dumps(res, indent=3))